# Run Dates Analysis
Try to install dates software

In [1]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp

### For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
### Make sure to have the font installed (it is on cluster for Harald)
rcParams['font.sans-serif'] = ['Arial']

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

compute-a-17-91.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer
CPU Count: 32
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


# Install
For install, do a git clone from Priyas git site,

git clone https://github.com/priyamoorjani/DATES

# Run DATES

### Modify the code here

In [ ]:
def qpAdm_run(leftpops, rightpops, output_file, 
              input_folder = "./eigenstrat/combined/", 
              input_file="punic.v44",
              par_file_folder = "./parfiles/", 
              input_ind_suff="_mod1", 
              output_folder ="./output/qpAdm/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=False):
    """Run qpAdm. Write temporary parfile and run the analysis
    leftpops: List of left populations
    rightpops: List of right populations
    input_folder: Where to find the input files
    par_file_folder: Folder of the parameterfile
    input_file: The name of the input file
    input_ind_suff: Suffix of .ind file. To allow modified populations
    input_ind: name of the input .ind file. If given - write over default"""
    #print(os.getcwd())
    parfile_path = par_file_folder + "parfile." + output_file
    left_path = par_file_folder + "left." + output_file
    right_path = par_file_folder + "right." + output_file
    
    ### Create the parfile:
    with open(parfile_path, 'w') as f:
        f.write("%s\n" % ("DIR: " + input_folder))
        f.write("%s\n" % ("S1: " + input_file))
        
        indline = "indivname: DIR/S1" + input_ind_suff + ".ind"
        f.write("%s\n" % indline)
        f.write("%s\n" % "snpname: DIR/S1.snp")
        f.write("%s\n" % "genotypename: DIR/S1.geno")
        f.write("%s\n" % ("popleft: " + left_path))
        f.write("%s\n" % ("popright: " + right_path))
        f.write("%s\n" % "details: YES")   
        if all_snps==True:
            f.write("%s\n" % "allsnps: YES")
    
    ### Write leftpops rightpops:       
    with open(left_path, 'w') as f:
        f.write("\n".join(leftpops))
        
    with open(right_path, 'w') as f:
        f.write("\n".join(rightpops))
    
    output_path = output_folder + output_file + ".log"
    command = f"{path_bin_qpAdm} -p {parfile_path} > {output_path}"
    #print(f"Running command: \n{command}")
    
    ### Run qpAdm
    start = time()
    #!$path_bin_qpAdm -p $parfile_path > $output_path
    os.system(command)
    end = time()
    
    print("Runtime: %2f" % (end - start))
    return 0

In [17]:
path_bin = "/n/groups/reich/hringbauer/o2bin/dates"
par_file = "/n/groups/reich/hringbauer/git/DATES/test_hr/par.dates"

In [19]:
%%time
!module load fftw/3.3.7; $path_bin -p $par_file


## DATES.  Version 3530

parameter file: /n/groups/reich/hringbauer/git/DATES/test_hr/par.dates
DIR: /n/groups/reich/hringbauer/git/DATES/test_hr/
indivname: /n/groups/reich/hringbauer/git/DATES/test_hr//family_packed.ind	
snpname: /n/groups/reich/hringbauer/git/DATES/test_hr//family_packed.snp
genotypename: /n/groups/reich/hringbauer/git/DATES/test_hr//family_packed.geno
admixlist: /n/groups/reich/hringbauer/git/DATES/test_hr//admix	
binsize: 0.001
maxdis: 1.0
seed: 77
runmode: 1
chithresh: 0.0
mincount: 1
zdipcorrmode: YES
qbin: 10
runfit: YES
afffit: YES
lovalfit: 0.45
usevar set!

seed: 77
genotypename:  /n/groups/reich/hringbauer/git/DATES/test_hr//family_packed.geno
packed geno read OK
end of inpack
valid snps: 378678
zzsnp: 248156 rs3781759     0.627147     0.445896
calling dowork:  French  Yoruba  Simulation 

French
Yoruba
admix: Simulation

countpops called
output tables: Simulation.out
mesh used :: qbin: 10  numqbins: 1404425
regdim: 2
number admixed: 1 numeg: 2
group 0    